# Segmenting and Clustering Neighborhoods in Toronto, Canada

## Part 1. In this part, we will explore and cluster the neighborhoods in Toronto

At first, we should get the data from wiki and prepare it for furher processing

In [2]:
##import required libraries
import pandas as pd
import numpy as np

In [3]:
##scrap data from wiki and prepare pandas dataframe 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

tables = pd.read_html(url)
df = tables[0][0:]
df.head(3)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods


#### Dataframe processing

In [5]:
#exclude rows with a borough that is Not assigned
df['Borough'].replace('Not assigned',np.nan, inplace=True)
df = df.dropna(axis='rows')

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [6]:
#If a cell has a borough but a Not assigned neighborhood, 
#then the neighborhood will be the same as the borough.
df['Neighbourhood'].replace('Not assigned',df['Borough'], inplace=True)
df.iloc[6,]

Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Queen's Park
Name: 8, dtype: object

In [7]:
#group neigborhoods by postcode and join them in one row separated by comma
#df_tst = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: "{%s}" % ', '.join(x))
df_tst = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_tst.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
#and we have..
df_tst.shape

(103, 3)

#### Now, it's time to add geographical coordinates to our dataframe

In [16]:
#load csv with geo coordinates
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
#geo_df.head(2)

#merge neighbourhoods dataframe with the one above
res_df = df_tst.set_index('Postcode').join(geo_df.set_index('Postal Code'))
res_df.tail(3)

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
M9W,Etobicoke,Northwest,43.706748,-79.594054


### Explore and cluster the neighborhoods in Toronto

At first, we will import additional libraries to create maps and cluster data

In [17]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


##### Create a map of Toronto with Neighbourhoods

In [22]:
latitude = 43.6532
longitude = -79.3832

# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(res_df['Latitude'], res_df['Longitude'], res_df['Borough'], res_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next, I'm going to simplify the above map and segment and cluster only the neighborhoods in **Noth York**. 

In [27]:
northYork_df = res_df[res_df['Borough'] == 'North York'].reset_index(drop=False)
northYork_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [34]:
# create map of Manhattan using latitude and longitude values
ny_latitude = 43.7615
ny_longitude = -79.4110
map_ny = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(northYork_df['Latitude'], northYork_df['Longitude'], northYork_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Exploratory analysis

In [30]:
#forsquare credentials
CLIENT_ID = 'BSG4WZUATSUUDOL2WSF22FDDJLXCIFYFYUMEVNGWE44011ES' # your Foursquare ID
CLIENT_SECRET = 'BIQ3J2INYQ4RTZ4G2AT5ZW24GT2OQOG3D22HRMMNKC0ISIOG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BSG4WZUATSUUDOL2WSF22FDDJLXCIFYFYUMEVNGWE44011ES
CLIENT_SECRET:BIQ3J2INYQ4RTZ4G2AT5ZW24GT2OQOG3D22HRMMNKC0ISIOG


Get top 10 venues in Noth York

In [40]:
LIMIT = 10 # limit of number of venues
radius = 500 # radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ny_latitude, 
    ny_longitude, 
    radius, 
    LIMIT)
url # display URL

results = requests.get(url).json()
#results

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
#clean the json and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Kinka Izakaya,Japanese Restaurant,43.760161,-79.409827
1,Sushi Moto Sake & Wine Bar,Sushi Restaurant,43.763902,-79.411559
2,Longo’s Sheppard Centre,Grocery Store,43.762221,-79.410762
3,EatBKK,Thai Restaurant,43.759320,-79.410454
4,Whole Foods Market,Grocery Store,43.760662,-79.411044


In [43]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Let's get Noth York venues

In [46]:
ny_venues = getNearbyVenues(names=northYork_df['Neighbourhood'],
                                   latitudes=northYork_df['Latitude'],
                                   longitudes=northYork_df['Longitude']
                                  )


Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [47]:
ny_venues.head(3)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool


let's check how many venues were returned for each neighborhood

In [49]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,3,3,3,3,3,3
Downsview Northwest,5,5,5,5,5,5
Downsview West,4,4,4,4,4,4
"Downsview, North Park, Upwood Park",5,5,5,5,5,5


In [50]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 79 uniques categories.


## Analize each neighborhood

In [54]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head(10)

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Shopping Mall,Steakhouse,Sushi Restaurant,Tea Room,Toy / Game Store,Vietnamese Restaurant
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Hillcrest Village,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [57]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped.head(3)

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Shopping Mall,Steakhouse,Sushi Restaurant,Tea Room,Toy / Game Store,Vietnamese Restaurant
0,"Bathurst Manor, Downsview North, Wilson Heights",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.0,...,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0
1,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.1,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0


#### Print each neighborhood along with the top 5 most common venues

In [58]:
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                       venue  freq
0                Coffee Shop   0.2
1                Bridal Shop   0.1
2  Middle Eastern Restaurant   0.1
3           Sushi Restaurant   0.1
4              Deli / Bodega   0.1


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1  Japanese Restaurant  0.25
2                 Café  0.25
3                 Bank  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0              Coffee Shop   0.2
1  Comfort Food Restaurant   0.1
2         Sushi Restaurant   0.1
3                     Café   0.1
4               Restaurant   0.1


----CFB Toronto, Downsview East----
                      venue  freq
0                   Airport  0.33
1         Other Repair Shop  0.33
2                      Park  0.33
3              Intersection  0.00
4  Mediterranean Restaurant  0.00


----Don Mills North----
                  venue 

## 4. Cluster Neighborhoods

In [60]:
#we will use k-means clustering

#number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 4, 0, 0, 0, 4, 4, 0], dtype=int32)

Let's create new df that includes cluster and top 10 venues for a neghborhood 

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [65]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(1)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Fast Food Restaurant,Restaurant,Middle Eastern Restaurant,Bridal Shop,Deli / Bodega,Bank,Diner,Sushi Restaurant,Dog Run


In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = northYork_df 

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

ny_merged.head() 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Fast Food Restaurant,Golf Course,Athletics & Sports,Dog Run,Mediterranean Restaurant,Pool,Convenience Store,Chinese Restaurant,Clothing Store,Coffee Shop
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Candy Store,Tea Room,American Restaurant,Shopping Mall,Salon / Barbershop,Toy / Game Store,Electronics Store,Bakery,Burger Joint,Movie Theater
2,M2K,North York,Bayview Village,43.786947,-79.385975,0,Chinese Restaurant,Café,Japanese Restaurant,Bank,Deli / Bodega,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1,Cafeteria,Vietnamese Restaurant,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,3,Piano Bar,Vietnamese Restaurant,Candy Store,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


## Create a map of clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters